In [1]:
import cv2
import lmdb
import random
import numpy as np
from tqdm import tqdm

In [16]:
# keys 列表存放图像和标签的地址，
data_list_fns = ['data_list/val.csv']

keys = []
for data_list_fn in data_list_fns:
    with open(data_list_fn, 'r') as f:
        lines = f.readlines() 
        keys += [k.replace('\\', '/') for line in lines for k in line.strip().split(', ')]

In [ ]:
keys

In [18]:
env = lmdb.open('small_dataset_lmdb', map_size=int(1e9)) 

数据库环境的结构：
一个环境可能包含多个数据库，所有数据库都驻留在同一个共享内存映射和底层磁盘文件中。
在同一个进程中同时打开同一个 LMDB 文件是一个严重的错误。 不注意这一点可能会导致数据损坏和解释器崩溃。

path:
存储数据库的目录位置（如果 subdir=True）或文件前缀。

map_size
最大的数据库可能会增长的大小。用于调整内存映射的大小。如果数据库增长大于“map_size”，将引发异常，用户必须关闭并重新打开环境。在 64 位上，这样做不会受到惩罚（比如 1TB）。 在 32 位上必须小于 2GB。
————————————————

原文链接：https://blog.csdn.net/Sonia8/article/details/123249240

数据集的lmdb：
将数据集转换成lmdb实际上就是一条条地将img和label的key-value形式写进lmdb中

img数据在lmdb中是以二进制形式存储的

In [22]:
txn = env.begin(write=True)
for key in tqdm(keys):
    with open(key, 'rb') as f:
        img_bytes = f.read() # 读取图片
    
    # 添加数据
    txn.put(key.encode(), img_bytes) 

# 通过commit()函数提交更改 
txn.commit()
env.close()

100%|██████████| 40/40 [00:00<00:00, 1246.39it/s]


In [ ]:
# test reading
env = lmdb.open('small_dataset_lmdb')
txn = env.begin(write=False)
# 打乱数据
random.shuffle(keys) 

for key in keys:
    img_bytes = txn.get(key.encode()) # 编码数据
    img_bytes = np.array(bytearray(img_bytes), dtype=np.uint8) # 数据类型转换成数组
    
    # 解码数据
    if key.endswith('.jpg'):
        img = cv2.imdecode(img_bytes, cv2.IMREAD_COLOR) 
    elif key.endswith('.png'):
        img = cv2.imdecode(img_bytes, cv2.IMREAD_GRAYSCALE)
    else:
        print('Unknown image type: %s' % key)
        exit()
    # 图片 resize
    img = cv2.resize(img, (img.shape[1]//3, img.shape[0]//3))
    # 显示图片
    cv2.imshow('show', img)
    key = cv2.waitKey(0)
    if 27 == key:
        break
env.close()